In [1]:
import pandas as pd
import numpy as np
import datetime
from pandasql import sqldf
import sqlite3
pd.options.display.max_rows = 50
pd.options.display.max_columns = 100

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

In [3]:
conn = sqlite3.connect('0-2DTE_spy_options_01Jan23-28Feb23v2_post_processed.db')
query_tables = 'SELECT * from "jan-feb23";'
df = pd.read_sql_query(query_tables, conn)
df['time_converted'] = pd.to_datetime(df['time_converted'])
df['expiry'] = pd.to_datetime(df['expiry'])

In [28]:
a = round(day_df['equity_start_price'].unique()[0])
b = 'put'
day_df = day_df.query(f'strike == {a} and type == "{b}"')
step = 1
central_strike = round(day_df['equity_start_price'].min())
print(central_strike)
strike_range = list(range(central_strike-3*step,central_strike+3*step,step))#
print(day_df['strike'].unique())
day_df1 = day_df[day_df['strike'].isin(strike_range)]
print(day_df['strike'].unique())

397
[397]
[397]


In [ ]:
# random seeds
rng = np.random.default_rng(42)
rand_dict = dict(zip(
    df['time_converted'].dt.date.unique().tolist(),
    rng.uniform(0.0,1.0,len(df['time_converted'].dt.date.unique())).tolist(),
))

# stop loss / take profit by DTE
# 0,1,2 - DTE_Adjusted. Profit/loss as abs pct. example 4 - 400% of entry price, so delta 300%, 0.5 loss - 50% of the bet
SL_TP_dict = {
    0 : {'profit' : 4.0, 'loss': 0.5},
    1 : {'profit' : 4.0, 'loss': 0.5},
    2 : {'profit' : 4.0, 'loss': 0.5},
}


for day in df['time_converted'].dt.date.unique():
  
    day_df = df[df['time_converted'].dt.date == day]
    
    # do some basic data control checks
    assert len(day_df['equity_start_price'].unique()) == 1, \
                'Equity start price is not unique for that date'

    strike_step = 1
    
    DTE_distinct = list(day_df['DTE_adjusted'].unique()) # used later to iterate across DTE
    if rand_dict[day]>0.5: # long
        
        
        central_strike = round(day_df['equity_start_price'].min())
        strike_range = list(range(central_strike-3*strike_step,central_strike+3*strike_step,strike_step))

        day_df = day_df[day_df['strike'].isin(strike_range)]        

        
        
        
        
        for DTE in DTE_distinct:
           
            # set tp/sl accordingly
            tp = SL_TP_dict[DTE]['profit']
            sl = SL_TP_dict[DTE]['loss']
            # get calls since we are long and then filter on specific DTE
            day_dte_df = day_df[(day_df['DTE_adjusted'] == DTE) & (day_df['type'] == 'Call')]

            # get specific strikes
            # for now only pickup ATM plus/minus 3 strikes. ATM is equity start price
            
            

    else: # sell
        pass   


In [4]:
logreg=LogisticRegression(random_state=42)


In [5]:
df.columns

Index(['volume_options', 'volume_weighted_options', 'open_options',
       'close_options', 'high_options', 'low_options', 'timestamp_options',
       'number of trades_options', 'ticker', 'full_name', 'type', 'strike',
       'expiry', 'equity_start_price', 'time_converted', 'open_equity',
       'close_equity', 'high_equity', 'low_equity', 'timestamp_equity',
       'number of trades_equity', '_merge', 'equity_pct_change',
       'options_pct_change', 'options_earliest_open',
       'equity_pct_change_normalized', 'day_classification', 'DTE', 'day_name',
       'DTE_adjusted'],
      dtype='object')

In [45]:
df_for_logreg = df[['day_classification','day_name','time_converted']]
df_for_logreg.loc[:,'time_converted'] = df_for_logreg.loc[:,'time_converted'].str.slice(0,11)
#df_for_logreg['time_converted'] = pd.to_datetime(df_for_logreg['time_converted'])
df_for_logreg.drop_duplicates(inplace=True)

C:\Users\dkarl\AppData\Local\Temp\ipykernel_7048\1902425135.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_for_logreg.drop_duplicates(inplace=True)


In [52]:
encoder = OneHotEncoder(sparse_output=False).set_output(transform="pandas")
ohe = encoder.fit_transform(df_for_logreg[['day_name']])
ohe
dd

,day_name_Friday,day_name_Monday,day_name_Thursday,day_name_Tuesday,day_name_Wednesday
0,0.0,0.0,0.0,1.0,0.0
2998,0.0,0.0,0.0,0.0,1.0
6238,0.0,0.0,1.0,0.0,0.0
9369,1.0,0.0,0.0,0.0,0.0
12520,0.0,1.0,0.0,0.0,0.0
15896,0.0,0.0,0.0,1.0,0.0
19103,0.0,0.0,0.0,0.0,1.0
22396,0.0,0.0,1.0,0.0,0.0
24780,1.0,0.0,0.0,0.0,0.0
27022,0.0,0.0,0.0,1.0,0.0


In [48]:
df_for_logreg


,day_classification,day_name,time_converted
0,high,Tuesday,2023-01-03
2998,average,Wednesday,2023-01-04
6238,very_low,Thursday,2023-01-05
9369,very_high,Friday,2023-01-06
12520,average,Monday,2023-01-09
15896,very_low,Tuesday,2023-01-10
19103,very_low,Wednesday,2023-01-11
22396,high,Thursday,2023-01-12
24780,high,Friday,2023-01-13
27022,very_low,Tuesday,2023-01-17


In [55]:
df['time_converted'] = pd.to_datetime(df['time_converted'])

array([datetime.date(2023, 1, 3), datetime.date(2023, 1, 4),
       datetime.date(2023, 1, 5), datetime.date(2023, 1, 6),
       datetime.date(2023, 1, 9), datetime.date(2023, 1, 10),
       datetime.date(2023, 1, 11), datetime.date(2023, 1, 12),
       datetime.date(2023, 1, 13), datetime.date(2023, 1, 17),
       datetime.date(2023, 1, 18), datetime.date(2023, 1, 19),
       datetime.date(2023, 1, 20), datetime.date(2023, 1, 23),
       datetime.date(2023, 1, 24), datetime.date(2023, 1, 25),
       datetime.date(2023, 1, 26), datetime.date(2023, 1, 27),
       datetime.date(2023, 1, 30), datetime.date(2023, 1, 31),
       datetime.date(2023, 2, 1), datetime.date(2023, 2, 2),
       datetime.date(2023, 2, 3), datetime.date(2023, 2, 6),
       datetime.date(2023, 2, 7), datetime.date(2023, 2, 8),
       datetime.date(2023, 2, 9), datetime.date(2023, 2, 10),
       datetime.date(2023, 2, 13), datetime.date(2023, 2, 14),
       datetime.date(2023, 2, 15), datetime.date(2023, 2, 16),
    